# Modelo predictivo

### Importar paquetes

In [7]:
import pandas as pd

### Limpieza de los datos

Empezaremos por todas las transformaciones necesarias tras el análisis previo. El objetivo es conseguir un conjunto de entrenamiento y test para nuestros modelos.

A pesar de que el reto específica que se nos proporcionarían diferentes conjuntos de test, tan solo contamos con uno. Eso nos hizo sospechar, tras un examen manual, nos dimos cuenta que los lotes de test están mezclados con el resto. Así que lo primero será separar dichos lotes y a partir de ahi construir la tabla con los campos relevantes. 

Este proceso será iterativo, por lo que complica la entrega, ya que solo podemos entregar un archivo. Por tanto se irá documentando, las diferentes transformaciones que se harán en distintos puntos del proceso hasta conseguir un modelo con buen rendimiento.

Como primer acercamiento será obtener los valores agregados de las distintas fases para poder predecir la variable objetivo.
Para ello necesitaremos crear un `pipeline` que garantize el orden de los pasos, dado que la entrada que se nos proporciona para el test es insuficiente para realizar una predicción

In [2]:
df_cultivo_test = pd.read_excel('raw_data/Fases producción_test v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo_test.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,24054,1,NaN,14616,2024-04-16 08:12:00,2024-04-18 07:28:00,81.6,15.44,85.60,184800000,14246.0,27.84,23.96,NaN,NaN
1,24055,1,NaN,14614,2024-04-13 08:18:00,2024-04-15 08:30:00,NaN,14.32,73.68,175200000,12912.0,30.96,23.16,NaN,NaN
2,24056,1,NaN,14615,2024-04-13 08:18:00,2024-04-15 08:15:00,NaN,14.56,82.40,168000000,14246.0,29.52,28.88,NaN,NaN
3,24057,1,NaN,13170,2024-04-16 08:12:00,2024-04-18 07:41:00,82.4,17.76,78.96,180800000,12912.0,31.04,25.32,NaN,NaN
4,24058,2,24055.0,14614,2024-04-15 12:28:00,2024-04-17 08:14:00,87.2,18.00,82.40,144800000,12912.0,26.08,20.36,NaN,NaN


In [3]:
lotes_test = df_cultivo_test['LOTE'].unique()
len(lotes_test)

56

In [4]:
df_cultivo = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,23019,1,NaN,14615,2023-03-21 07:30:00,2023-03-23 06:30:00,82.4,17.28,91.2,184000000,17825,NaN,NaN,1747.920,6.00
1,23020,1,NaN,14616,2023-03-21 07:30:00,2023-03-23 06:30:00,80.4,18.80,91.2,181600000,14246,NaN,NaN,1676.160,6.56
2,23021,1,NaN,13170,2023-03-22 07:30:00,2023-03-24 06:30:00,66.4,16.16,86.4,248000000,17825,NaN,NaN,1928.496,8.08
3,23022,1,NaN,14614,2023-03-22 07:30:00,2023-03-24 06:30:00,85.6,18.48,83.2,229600000,12912,NaN,NaN,1782.800,5.92
4,23023,1,NaN,14615,2023-03-28 07:27:00,2023-03-30 10:00:00,77.6,17.12,74.4,132800000,17825,26.56,20.88,1861.840,2.96


___

In [38]:
def get_OF_dataset():
    # Cargamos datos
    df = pd.read_excel('raw_data/OF 123456 v02.xlsx', engine='openpyxl')

    # Corregimos caracteres especiales en Lote 
    df['LOTE'] = df['Lote'].apply(lambda x: int(x.replace('/', '').replace('P', '')))

    # Renombrado de columnas
    df = df.rename(columns={'Orden': 'orden', 'Cantidad entregada': 'cantidad'})
    
    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['orden', 'cantidad']

    return df[final_cols]

In [39]:
def get_preinoculo_dataset(lotes):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Preinóculo', header=[0, 1], na_values=['NA', 'N.A'], engine='openpyxl')
    # Corregir cabeceras
    new_cols = [l2 if 'Unnamed' in l1 else f"{l1}-{l2}" for l1, l2 in df.columns.to_list()]
    df.columns = new_cols

    # Filtro de lotes
    df = df[df['LOTE'].isin(lotes)]

    # Imputación valores perdidos por la media
    for c in df.select_dtypes(include='float').columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(df[c].mean())
        
    # Valores de pH y turbidez seleccionados para siguiente fase
    ph_cols = [col for col in df.columns if col.startswith('pH')]
    turbidez_cols = [col for col in df.columns if col.startswith('Turbidez')]
    lineas_cols = [col for col in df.columns if col.startswith('Línea')]

    # el ph y turbidez finales las consideramos como la mezcla de las dos líneas escogidas
    df['ph'] = (
        (
            df[ph_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(3)
    
    df['turbidez'] = (
        (
            df[turbidez_cols].values *
            df[lineas_cols].values
        ).sum(axis=1) /
        df[lineas_cols].sum(axis=1)
    ).round(2)

    # Calculo duración
    # 1. obtenemos las horas
    # 2. corregimos las fechas "al reves"
    # 3. corregimos las horas negativas
    ini_col = 'Fecha/hora inicio'
    fin_col = 'Fecha/hora fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['ph','turbidez','duracion_horas']

    return df[final_cols]


In [46]:
def get_inoculo_dataset(lotes, df_preinoculo):
    # Carga de datos
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Inóculo', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')

    # Filtrado de lotes 
    df = df[df['LOTE'].isin(lotes)]
    df = df.dropna(subset=['Fecha/hora inicio','Fecha/hora fin'])

    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 
                            'Fecha/hora fin': 'ts_fin', 
                            'ID bioreactor': 'id_bioreactor',
                            'Turbidez inicio cultivo': 'turbidez_ini',
                            'Turbidez final culttivo': 'turbidez_fin'}
    )

    # Imputación de valores perdidos
    df.loc[df['turbidez_ini'].isna(), ['turbidez_ini']] = (
        df[df['turbidez_ini'].isna()]
        .join(df_preinoculo, 
              on='LOTE', how='left', rsuffix='preinoculo')
        ['turbidez']
    )
    df['turbidez_dif'] = df['turbidez_fin'] - df['turbidez_ini']

    # > Si con la anterior imputación volvemos a tener un NaN en la resta, imputamos por la media
    df['turbidez_dif'] = df['turbidez_dif'].fillna(df['turbidez_dif'].mean())
    
    # Calculo duracion de la fase en horas
    ini_col = 'ts_inicio'
    fin_col = 'ts_fin'
    df['duracion_horas'] = (df[fin_col] - df[ini_col]).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, [ini_col, fin_col]] = df.loc[df['duracion_horas'] < 0, [fin_col, ini_col]].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)

    # LOTE como indice y variables escogidas
    df = df.set_index('LOTE')
    final_cols = ['id_bioreactor', 'ts_inicio', 'ts_fin', 'turbidez_dif', 'duracion_horas']

    return df[final_cols]

In [47]:
df_info_general = get_OF_dataset()
lotes = df_info_general.index.unique()
df_preinoculo = get_preinoculo_dataset(lotes)
df_inoculo = get_inoculo_dataset(lotes, df_preinoculo)

In [48]:
df_inoculo

,id_bioreactor,ts_inicio,ts_fin,turbidez_dif,duracion_horas
LOTE,,,,,
23023,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
23024,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
23025,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
23026,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
23027,13172,2023-04-03 13:30:00,2023-04-04 11:35:00,4.64,22.08
...,...,...,...,...,...
24101,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
24103,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
24104,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,11.92,25.00


In [150]:
get_inoculo_dataset(lotes, preinoculo)

,LOTE,id_bioreactor,ts_inicio,ts_fin,diferencia_turbidez,duracion_horas
4,23023,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
5,23024,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
6,23025,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
7,23026,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
8,23027,13172,2023-04-03 13:30:00,2023-04-04 11:35:00,4.64,22.08
...,...,...,...,...,...,...
163,24101,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
164,24103,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
165,24104,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,11.92,25.00
166,24105,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,11.92,25.00
